In [35]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rv_discrete
from random import choices

In [45]:
def unif_sample(n, k, method = 'replacement'):
    if method == 'replacement':
        ind = np.array(choices(range(n), k=k))
    elif method == 'noreplacement':
        ind = np.random.permutation(range(n))[:k]
    elif method == 'bernoulli':
        rv = rv_discrete(values=((0,1), (1-k/n, k/n)))
        ind = np.argwhere(rv.rvs(size=(n,))).flatten()

    return ind



In [46]:
from scipy.sparse.linalg import LinearOperator, lsqr
def sketch_and_precondition(A, b, ind, cond = True):
        m = A.shape[0]
        d = ind.size
        SA = np.sqrt(m/d)*A[ind,:]
        q, r = np.linalg.qr(SA, mode='reduced')

        Ap = LinearOperator(shape = A.shape, \
                            matvec = lambda x: A @ np.linalg.solve(r, x), \
                            rmatvec = lambda x: np.linalg.solve(r.T, A.T @ x), dtype = 'd')
        
        if cond:
                print("Prec cond number", np.linalg.cond(np.linalg.solve(r.T, A.T).T))

        res = lsqr(Ap,b)
        print('The number of preconditioned LSQR iterations is ',res[2])
        return np.linalg.solve(r, res[0])

### Ill-conditioned case, with replacement

In [47]:
m = 10**3; n = 50 
A = np.random.randn(m, n) @ np.diag(0.8**np.arange(n))
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))
 
# Small residual
b = A @ xt + 1.e-6*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]

print('Small residual case.')
ind = unif_sample(m, 3*n, 'replacement')
xs= sketch_and_precondition(A, b, ind)
print("Relative error is ", np.linalg.norm(xs-xls)/np.linalg.norm(xls))

res = lsqr(A,b)
print('The number of unpreconditioned LSQR iterations is ',res[2])

print("\n")

# Large residual
b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]

print("Large residual case.")
ind = unif_sample(m, 2*n, 'replacement')
xs= sketch_and_precondition(A, b, ind)
print("Relative error in computed solutions", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
res = lsqr(A,b)
print('The number of unpreconditioned LSQR iterations is ',res[2])

Condition number of A 59366.46151000063
Small residual case.
Prec cond number 3.262316573634254
The number of preconditioned LSQR iterations is  17
Relative error is  0.011603532462770034
The number of unpreconditioned LSQR iterations is  100


Large residual case.
Prec cond number 5.039286550304222
The number of preconditioned LSQR iterations is  31
Relative error in computed solutions 1.0204216332837246e-05
The number of unpreconditioned LSQR iterations is  100


### Comparing different sampling methods

In [51]:
m = 10**3; n = 50 
A = np.random.randn(m, n) @ np.diag(0.8**np.arange(n))
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))


# Large residual
b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]

ind = unif_sample(m, 2*n, 'replacement')
xs= sketch_and_precondition(A, b, ind)
print("Relative error (replacement) is", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
ind = unif_sample(m, 2*n, 'noreplacement')
xs= sketch_and_precondition(A, b, ind)
print("Relative error (noreplacement) is", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
ind = unif_sample(m, 2*n, 'bernoulli')
xs= sketch_and_precondition(A, b, ind)
print("Relative error (bernoulli) is", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
res = lsqr(A,b)
print('The number of unpreconditioned LSQR iterations is ',res[2])

Condition number of A 58366.73008999272
Prec cond number 6.129312295202938
The number of preconditioned LSQR iterations is  33
Relative error (replacement) is 1.2611738092342175e-05
Prec cond number 4.668834938550401
The number of preconditioned LSQR iterations is  30
Relative error (noreplacement) is 1.0299500978667102e-05
Prec cond number 5.397927612469492
The number of preconditioned LSQR iterations is  34
Relative error (bernoulli) is 9.197520454244763e-06
The number of unpreconditioned LSQR iterations is  100


### Effect of oversampling

In [91]:
m = 10**3; n = 50 
A = np.random.randn(m, n) @ np.diag(0.8**np.arange(n))
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))


b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]
for d in [n, int(1.2*n), int(1.5*n), 2*n]:
    ind = unif_sample(m, d, 'replacement')
    print("Number of samples:", d)
    xs= sketch_and_precondition(A, b, ind)
    print("Relative error:", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
    print("\n")

Condition number of A 59085.54908391729
Number of samples: 50
Prec cond number 1.609344547070061e+18
The number of preconditioned LSQR iterations is  2
Relative error: 414.2251670231169


Number of samples: 60
Prec cond number 20.7175216724068
The number of preconditioned LSQR iterations is  52
Relative error: 3.64807679387587e-05


Number of samples: 75
Prec cond number 8.00862787181412
The number of preconditioned LSQR iterations is  41
Relative error: 2.361735816815901e-05


Number of samples: 100
Prec cond number 6.4119115831354865
The number of preconditioned LSQR iterations is  33
Relative error: 1.4393940196372111e-05




### Pathological case: High coherence

In [92]:
m = 10**3; n = 50 
A = np.zeros((m,n), dtype ='d'); A[:n, :] = np.eye(n)
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))


b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]
for d in [n,  2*n, 3*n]:
    ind = unif_sample(m, d, 'replacement')
    print("Number of samples:", d)
    SA = np.sqrt(m/d)*A[ind,:]
    print('Condition number of SA:', np.linalg.cond(SA))
    print("\n")

Condition number of A 1.0
Number of samples: 50
Condition number of SA: inf


Number of samples: 100
Condition number of SA: inf


Number of samples: 150
Condition number of SA: inf




### Pathological case: with Gaussians

In [65]:
m = 10**3; n = 50 
A = np.zeros((m,n), dtype ='d'); A[:n, :] = np.eye(n)
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))


b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]
for d in [n, int(1.2*n), int(1.5*n), 2*n, 3*n]:
    S = np.random.randn(d,m)/np.sqrt(d)
    print("Number of samples:", d)
    SA =S @A
    print('Condition number of SA:', np.linalg.cond(SA))
    print("\n")

Condition number of A 1.0
Number of samples: 50
Condition number of SA: 73.63757584103615


Number of samples: 60
Condition number of SA: 14.998776878811622


Number of samples: 75
Condition number of SA: 9.775729107327754


Number of samples: 100
Condition number of SA: 4.768392718181062


Number of samples: 150
Condition number of SA: 3.168044714807465




### Blendenpik: simplified implementation

In [86]:
from scipy.fftpack import dct

def blendenpik(A, b, d, cond=True):
    m = A.shape[0]
    ind = unif_sample(m, d)

    rv = rv_discrete(values=((-1,1), (0.5, 0.5)))
    D = np.diag(rv.rvs(size=(m,)))
    SA = np.sqrt(m/d)*dct(D @ A, axis = 0, norm='ortho')[ind,:]

    q, r = np.linalg.qr(SA, mode='reduced')

    Ap = LinearOperator(shape = A.shape, \
                        matvec = lambda x: A @ np.linalg.solve(r, x), \
                        rmatvec = lambda x: np.linalg.solve(r.T, A.T @ x), dtype = 'd')
        
    if cond:
        print("Prec cond number", np.linalg.cond(np.linalg.solve(r.T, A.T).T))

    res = lsqr(Ap,b)
    print('The number of Blendenpik iterations is ',res[2])
    return np.linalg.solve(r, res[0])

In [87]:
m = 10**3; n = 50 
A = np.random.randn(m, n) @ np.diag(0.8**np.arange(n))
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))
 
# Small residual
b = A @ xt + 1.e-6*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]

print('Small residual case.')
ind = unif_sample(m, 3*n, 'replacement')
xs= blendenpik(A, b, 3*n)
print("Relative error is ", np.linalg.norm(xs-xls)/np.linalg.norm(xls))

res = lsqr(A,b)
print('The number of unpreconditioned LSQR iterations is ',res[2])

print("\n")

# Large residual
b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]

print("Large residual case.")
xs= blendenpik(A, b, 3*n)
print("Relative error in computed solutions", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
res = lsqr(A,b)
print('The number of unpreconditioned LSQR iterations is ',res[2])

Condition number of A 57011.56074456341
Small residual case.
Prec cond number 3.330420263207191
The number of Blendenpik iterations is  17
Relative error is  0.01874599243490038
The number of unpreconditioned LSQR iterations is  100


Large residual case.
Prec cond number 3.803518783862722
The number of Blendenpik iterations is  27
Relative error in computed solutions 4.919630590481288e-06
The number of unpreconditioned LSQR iterations is  100


In [90]:
### Pathological case: Blendenpik

In [88]:
m = 10**3; n = 50 
A = np.zeros((m,n), dtype ='d'); A[:n, :] = np.eye(n)
xt = np.ones((n,))
print("Condition number of A", np.linalg.cond(A))


b = A @ xt + 1.e-3*np.random.randn(m)
res = np.linalg.lstsq(A, b, rcond = None)
xls = res[0]
rls = res[1]
for d in [n, int(1.2*n), int(1.5*n), 2*n, 3*n]:
    print('Sample size is', d)
    xs = blendenpik(A, b, d)
    print("Relative error:", np.linalg.norm(xs-xls)/np.linalg.norm(xls))
    print("\n")

Condition number of A 1.0
Sample size is 50
Prec cond number 4.4801932441008133e+18
The number of Blendenpik iterations is  3
Relative error: 0.95606978643339


Sample size is 60
Prec cond number 10850.752123156004
The number of Blendenpik iterations is  37
Relative error: 0.013027953745579276


Sample size is 75
Prec cond number 702.5093043443579
The number of Blendenpik iterations is  49
Relative error: 3.3979598022513443e-06


Sample size is 100
Prec cond number 1124.6387746982623
The number of Blendenpik iterations is  47
Relative error: 5.60708668241052e-06


Sample size is 150
Prec cond number 12.185698845444902
The number of Blendenpik iterations is  24
Relative error: 3.9231512370240596e-08




In [40]:
def leverage_scores(A):
    q, r = np.linalg.qr(A, mode='reduced')
    ls = np.apply_along_axis(lambda x: np.linalg.norm(x)**2., 1, q)
    rv = rv_discrete(values=(range(ls.size), ls/n))
    return rv

rv = leverage_scores(A)
print(rv.rvs(size=5))

[420 330 280 999 136]
